In [4]:
# pip install torch 
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch.nn as nn
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Import data
# https://www.kaggle.com/datasets/yashchoudhary/realsr-v3/data
# https://github.com/ngchc/CameraSR (City100)
# https://github.com/xiezw5/Component-Divide-and-Conquer-for-Real-World-Image-Super-Resolution?tab=readme-ov-file

# First, pick the 2000 image pairs and have two folders (train and test) and in each are 2 more folders (hr and lr). 
# .   
# ├── train
# │   ├── HR
# │   └── LR
# └── test
#     ├── HR
#     └── LR
# Pre-split already, 80-20 split

In [35]:
import os

DATASET_IMAGES_DIR = "./images" # Path to your directory where the images/ folder is 
CROPPED_DATASET_IMAGES_DIR = "./cropped_images" # Path to an empty directory for the cropped images

os.makedirs("{}/train/HR".format(CROPPED_DATASET_IMAGES_DIR), exist_ok=True)
os.makedirs("{}/test/HR".format(CROPPED_DATASET_IMAGES_DIR), exist_ok=True)
os.makedirs("{}/train/LR".format(CROPPED_DATASET_IMAGES_DIR), exist_ok=True)
os.makedirs("{}/test/LR".format(CROPPED_DATASET_IMAGES_DIR), exist_ok=True)


Randomly crop the images and store them

In [36]:
import PIL
from PIL import Image
import torchvision.transforms as T
import torch
from os import listdir

# Random crop transformation to apply
transform = T.Compose([
        T.RandomCrop(128)
])		

directories = ["test", "train"]
for directory in directories:
	# Deal with transforming both the LR and HR images at the same time in order 
	# to transform them the same way
	HR_dir = sorted(os.listdir("{}/{}/HR".format(DATASET_IMAGES_DIR, directory)))
	LR_dir = sorted(os.listdir("{}/{}/LR".format(DATASET_IMAGES_DIR, directory)))
	for imageIndex,img in enumerate(LR_dir):

		imgLR = Image.open("{}/{}/LR/".format(DATASET_IMAGES_DIR, directory)+LR_dir[imageIndex])
		imgHR = Image.open("{}/{}/HR/".format(DATASET_IMAGES_DIR, directory)+HR_dir[imageIndex])

		# Save the state and reload the rng state so that the same transformation is applied
		# to both images
		state = torch.get_rng_state()
		imgLR = transform(imgLR)
		torch.set_rng_state(state)
		imgHR = transform(imgHR)

		# Save the images in the cropped dataset folder
		imgLR.save("{}/{}/LR/".format(CROPPED_DATASET_IMAGES_DIR, directory)+LR_dir[imageIndex])
		imgHR.save("{}/{}/HR/".format(CROPPED_DATASET_IMAGES_DIR, directory)+HR_dir[imageIndex])

In [ ]:
# Create a Dataset class
# Code used from: https://discuss.pytorch.org/t/how-can-i-read-a-image-pair-at-the-same-time/23151
class MyDataset(Dataset):
    def __init__(self, lr_paths_dir, hr_paths_dir):
        self.lr_paths = lr_paths # List of image paths: ['./train/LR/image_1.bmp', './train/input/image_2.bmp', ...]
        self.hr_paths = hr_paths
        
    def __getitem__(self, index):
        lr_img = Image.open(self.lr_paths[index])
        hr_img = Image.open(self.hr_paths[index])
        return lr_img, hr_img
    
    def __len__(self):
        return len(self.lr_paths)

In [ ]:
train_set = MyDataset(
    sorted(os.listdir("{}/{}/LR".format(CROPPED_DATASET_IMAGES_DIR, "train"))), sorted(os.listdir("{}/{}/HR".format(CROPPED_DATASET_IMAGES_DIR, "train"))))

test_set = MyDataset(
    sorted(os.listdir("{}/{}/LR".format(CROPPED_DATASET_IMAGES_DIR, "test"))), sorted(os.listdir("{}/{}/HR".format(CROPPED_DATASET_IMAGES_DIR, "test"))))

In [ ]:
# Create the training and validation indices to split the train_set
dataset_size = len(train_set)
indices = list(range(dataset_size))
np.random.shuffle(indices)
split = int(len(indices) * 0.8) #split at 80%

train_indices, val_indices = indices[:split], indices[split:]
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
# Create a Dataloader for each train, val, and test
train_dl = DataLoader(train_set, batch_size=batch_size, 
											num_workers=1, sampler=train_sampler)
val_dl = DataLoader(train_set, batch_size=batch_size,
                      num_workers=1, sampler=val_sampler)
test_dl = DataLoader(test_set, batch_size=batch_size,
                      num_workers=1)
